In [1]:
import numpy as np 
import time

import torch 
import torch.nn as nn
import torch.nn.functional as F 

import matplotlib.pyplot as plt
from collections import OrderedDict

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.preprocessing import RobustScaler

from utils.inference import Trainer, plot_loss
from utils.models import DNN
import utils.datasets as d

# With sampling

In [2]:
X = np.vstack((np.load("data/sequenced_data_for_VAE_length-160_stride-10_pt1.npy"),
               np.load("data/sequenced_data_for_VAE_length-160_stride-10_pt2.npy")))
y = np.load("data/sequenced_data_for_VAE_length-160_stride-10_targets.npy")

In [3]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=0.2, random_state=666)

In [4]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=666)

In [6]:
train = d.H_alphaSequences(X_train, y_train)
valid = d.H_alphaSequences(X_valid, y_valid)
test = d.H_alphaSequences(X_test, y_test)

In [7]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=512, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=512, shuffle=True)

## Loading Encoder

In [8]:
VAE = torch.load("models_and_losses/DeepDenseVAE_mark_V_[160-256-128-15]_VDO_GNLL_400ep_lr-1e-4.pt")

In [9]:
VAE.encoder

Sequential(
  (0): Linear(in_features=160, out_features=256, bias=True)
  (1): ELU(alpha=1.0)
  (2): Linear(in_features=256, out_features=128, bias=True)
  (3): ELU(alpha=1.0)
  (4): VariationalLayer(
    (mu): Linear(in_features=128, out_features=15, bias=True)
    (rho): Linear(in_features=128, out_features=15, bias=True)
    (softplus): Softplus(beta=1, threshold=20)
  )
)

## Freezing Encoder

In [10]:
class Encoder_Classifier(nn.Module):
    def __init__(self, encoder, clf):
        super(Encoder_Classifier, self).__init__()
        self.encoder=encoder
        self.clf = clf
        self.frozen_encoder = False

    def freeze_encoder(self):
        for param in self.encoder.parameters():
            param.requires_grad = False
        self.frozen_encoder = True
        print("Encoder frozen.")

    def unfreeze_encoder(self):
        for param in self.encoder.parameters():
            param.requires_grad = True
        self.frozen_encoder = False
        print("Encoder unfrozen.")
    
    def forward(self, X):
        Z, mu, sigma = self.encoder(X)
        return self.clf(Z)
    

In [11]:
DNN1 = nn.Sequential(
            nn.Linear(in_features=15, out_features=64),
            nn.Sigmoid(),
            nn.Linear(in_features=64, out_features=128),
            nn.Sigmoid(),
            nn.Linear(in_features=128, out_features=4),
)

In [12]:
model = Encoder_Classifier(encoder=VAE.encoder, clf = DNN1)
model.freeze_encoder()

Encoder frozen.


In [13]:
optimizer= torch.optim.Adam(model.parameters(), lr=1e-2)

In [14]:
m1 = Trainer(
        model=model,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30], gamma=0.1),
        tensorboard=True,
        model_name="DNN_on_DeepDenseVAE_mark_V_sampling_latent_space_m.IV_lr-{1e-2,1e-3}_epochs-512",
        verbose=True
        )

cuda:0


In [15]:
lh = m1(epochs=100, train_loader=train_loader, validation_loader=valid_loader)

C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\jit\__init__.py:1007: TracerWarning: Trace had nondeterministic nodes. Did you forget call .eval() on your model? Nodes:
	%eps : Float(512, 15) = aten::randn_like(%sigma, %58, %59, %60, %61), scope: Encoder_Classifier/Sequential[encoder]/VariationalLayer[4] # Z:\Semi-supervised-Learning-with-VAE\layers.py:35:0
This may cause errors in trace checking. To disable trace checking, pass check_trace=False to torch.jit.trace()
  check_tolerance, _force_outplace, True, _module_class)
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\jit\__init__.py:1007: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Not within tolerance rtol=1e-05 atol=1e-05 at input[464, 1] (1.4070541858673096 vs. 1.3884215354919434) and 2040 other locations (99.00%)
  check_tolerance, _force_outplace, True, _module_class)
C:\Users\ghost_000\Miniconda3\envs\Py

Epoch [1/100], average_loss:0.7999, validation_loss:0.6471, val_accuracy:0.6753
Epoch [2/100], average_loss:0.5995, validation_loss:0.5644, val_accuracy:0.8442
Epoch [3/100], average_loss:0.5408, validation_loss:0.5181, val_accuracy:0.8182
Epoch [4/100], average_loss:0.4998, validation_loss:0.4760, val_accuracy:0.9091
Epoch [5/100], average_loss:0.4632, validation_loss:0.4509, val_accuracy:0.8312
Epoch [6/100], average_loss:0.4440, validation_loss:0.4409, val_accuracy:0.8312
Epoch [7/100], average_loss:0.4305, validation_loss:0.4262, val_accuracy:0.8442
Epoch [8/100], average_loss:0.4212, validation_loss:0.4220, val_accuracy:0.8052
Epoch [9/100], average_loss:0.4167, validation_loss:0.4179, val_accuracy:0.7662
Epoch [10/100], average_loss:0.4104, validation_loss:0.4154, val_accuracy:0.7922
Epoch [11/100], average_loss:0.4005, validation_loss:0.4094, val_accuracy:0.8312
Epoch [12/100], average_loss:0.3969, validation_loss:0.3977, val_accuracy:0.9091
Epoch [13/100], average_loss:0.3908, 

In [16]:
X_test = np.load("data/encoded_data_test_DeepDenseVAE_mark_V_[160-256-128-15]_VDO_GNLL_400ep_lr-1e-4.npy")
y_test = np.load("data/labels_test.npy")

In [17]:
#m1.tb.close()

m1.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m1.model.clf(torch.tensor(X_test).float().to(m1.device)).cpu().detach(), axis=1)

In [18]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.726923314423648

In [19]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.7635635755258127

In [20]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[2481,  947,   44,  763],
       [1369, 8473,  102,  501],
       [  14,   23,  425,   17],
       [  72,   85,   20, 1400]], dtype=int64)

In [21]:
#torch.save(m1.model.clf, "models_and_losses/DeepDenseVAE_mark_V_[160-256-128-15]_VDO_GNLL_400ep_lr-1e-4+DNN_on_VAE_sampling_latent_space_m.IV_lr-{1e-2,1e-3}_bs-512_epoch-100.pt")

C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sigmoid. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [22]:
lh = m1(epochs=range(100,200), train_loader=train_loader, validation_loader=valid_loader)

Epoch [101/200], average_loss:0.3216, validation_loss:0.3534, val_accuracy:0.8571
Epoch [102/200], average_loss:0.3222, validation_loss:0.3537, val_accuracy:0.8442
Epoch [103/200], average_loss:0.3206, validation_loss:0.3580, val_accuracy:0.8312
Epoch [104/200], average_loss:0.3225, validation_loss:0.3601, val_accuracy:0.7662
Epoch [105/200], average_loss:0.3210, validation_loss:0.3504, val_accuracy:0.8831
Epoch [106/200], average_loss:0.3218, validation_loss:0.3579, val_accuracy:0.7792
Epoch [107/200], average_loss:0.3212, validation_loss:0.3607, val_accuracy:0.7662
Epoch [108/200], average_loss:0.3223, validation_loss:0.3564, val_accuracy:0.8052
Epoch [109/200], average_loss:0.3201, validation_loss:0.3581, val_accuracy:0.8182
Epoch [110/200], average_loss:0.3198, validation_loss:0.3559, val_accuracy:0.8701
Epoch [111/200], average_loss:0.3211, validation_loss:0.3549, val_accuracy:0.8701
Epoch [112/200], average_loss:0.3200, validation_loss:0.3545, val_accuracy:0.8571
Epoch [113/200],

In [27]:
m1.tb.close()

m1.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m1.model.clf(torch.tensor(X_test).float().to(m1.device)).cpu().detach(), axis=1)

In [28]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.7373366438327096

In [29]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.7767686424474187

In [30]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[2623,  954,   51,  607],
       [1289, 8574,  121,  461],
       [  11,   20,  431,   17],
       [  93,   88,   24, 1372]], dtype=int64)

In [31]:
#torch.save(m1.model.clf, "models_and_losses/DeepDenseVAE_mark_V_[160-256-128-15]_VDO_GNLL_400ep_lr-1e-4+DNN_on_VAE_sampling_latent_space_m.IV_lr-{1e-2,1e-3}_bs-512_epoch-200.pt")

# bs 128

In [32]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=512, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=512, shuffle=True)

In [71]:
DNN2 = nn.Sequential(
            nn.Linear(in_features=15, out_features=64),
            nn.PReLU(),
            nn.Linear(in_features=64, out_features=64),
            nn.PReLU(),
            nn.Linear(in_features=64, out_features=4),
)

In [72]:
model2 = Encoder_Classifier(encoder=VAE.encoder, clf = DNN2)
model2.freeze_encoder()

Encoder frozen.


In [73]:
optimizer= torch.optim.Adam(model2.parameters(), lr=1e-2)

In [74]:
m2 = Trainer(
        model=model2,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30,60], gamma=0.1),
        tensorboard=True,
        model_name="DNN_on_VAE_sampling_latent_space_m.VI_lr-{1e-2,1e-3}_bs-512",
        verbose=True
        )

cuda:0


In [75]:
lh = m2(epochs=100, train_loader=train_loader, validation_loader=valid_loader)

C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\jit\__init__.py:1007: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Not within tolerance rtol=1e-05 atol=1e-05 at input[373, 2] (-1.1733821630477905 vs. -1.953189730644226) and 2046 other locations (99.00%)
  check_tolerance, _force_outplace, True, _module_class)
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\jit\__init__.py:1007: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Not within tolerance rtol=1e-05 atol=1e-05 at input[394, 2] (-3.9332759380340576 vs. -2.640728712081909) and 2046 other locations (99.00%)
  check_tolerance, _force_outplace, True, _module_class)
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\jit\__init__.py:1007: TracerWarning: Output nr 1. of the traced function does not match the correspon

Epoch [1/100], average_loss:0.5910, validation_loss:0.4819, val_accuracy:0.7792
Epoch [2/100], average_loss:0.4567, validation_loss:0.4272, val_accuracy:0.8961
Epoch [3/100], average_loss:0.4234, validation_loss:0.4245, val_accuracy:0.7662
Epoch [4/100], average_loss:0.4010, validation_loss:0.3996, val_accuracy:0.8571
Epoch [5/100], average_loss:0.3898, validation_loss:0.3841, val_accuracy:0.8312
Epoch [6/100], average_loss:0.3830, validation_loss:0.3864, val_accuracy:0.8571
Epoch [7/100], average_loss:0.3763, validation_loss:0.4091, val_accuracy:0.8701
Epoch [8/100], average_loss:0.3715, validation_loss:0.3807, val_accuracy:0.8831
Epoch [9/100], average_loss:0.3637, validation_loss:0.3912, val_accuracy:0.7532
Epoch [10/100], average_loss:0.3639, validation_loss:0.3792, val_accuracy:0.8052
Epoch [11/100], average_loss:0.3599, validation_loss:0.3810, val_accuracy:0.8312
Epoch [12/100], average_loss:0.3550, validation_loss:0.3751, val_accuracy:0.8052
Epoch [13/100], average_loss:0.3547, 

In [76]:
#lh = m2(epochs=range(100,200), train_loader=train_loader, validation_loader=valid_loader)

In [77]:
#m2.tb.close()
m2.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m2.model.clf(torch.tensor(X_test).float().to(m2.device)).cpu().detach(), axis=1)

In [78]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.733158548669584

In [79]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.7615320267686424

In [80]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[2675,  883,   28,  649],
       [1628, 8232,  115,  470],
       [  24,   14,  416,   25],
       [  61,   76,   18, 1422]], dtype=int64)

In [82]:
m2.tb.close()

In [83]:
#torch.save(m2.model.clf, "models_and_losses/DeepDenseVAE_mark_V_[160-256-128-15]_VDO_GNLL_400ep_lr-1e-4+DNN_(drop)_on_VAE_sampling_latent_space_m.VI_lr-{1e-2,1e-3}_bs-512_epoch-200.pt")

C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Dropout. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
